## Tensors(张量)

正如名称所示，TensorFlow是定义和运行张量的计算的框架。张量是将向量和矩阵推广到潜在的更高维度。在内部，TensorFlow将张量表示为基本数据类型的n维数组。

在编写TensorFlow程序时，您操作并传递的主要对象是`tf.Tensor`。 `tf.Tensor`对象表示将最终生成值的部分定义的计算。 TensorFlow程序首先构建tf.Tensor对象的图形，详细说明如何基于其他可用张量计算每个张量，然后通过运行该图的一部分来实现预期结果。

tf.Tensor具有以下属性：

1. 数据类型 dtype （例如，float32，int32或 string）.
2. 形状 shape.

张量中的每个元素都具有相同的数据类型，并且数据类型总是已知的。形状（即它具有的维度数量和每个维度的大小）可能只是部分已知的。如果其输入的形状也是完全已知的，则大多数操作会产生完全已知形状的张量，但在某些情况下，只能在图形执行时得到张量的形状。

某些类型的张量是特殊的。主要的是：

- tf.Variable
- tf.constant
- tf.placeholder
- tf.SparseTensor

除了`tf.Variable`之外，张量的值是不变的，这意味着在单个执行张量的情况下只有一个值。然而，两次运行相同的张量可能会返回不同的值; 例如张量可以是从磁盘读取数据或生成随机数的结果。

### Rank(秩/维度)

tf.Tensor对象的秩是它的维数。秩的同义词包括**order**或**degree**或**n-dimension**。请注意，TensorFlow中的秩与数学中的矩阵秩并不相同。如下表所示，TensorFlow中的每个秩都对应一个不同的数学实体：

| Rank | Math Entity |
|------|-------------|
| 0    | Scalar (magnitude only) 标量(只有大小) |
| 1    | Vector (magnitude and direction) |
| 2    | Matrix (table of numbers)  |
| 3    | 3-Tensor (cube of numbers)  |
| n    | 以此类推 |

### Rank 0

以下片段演示了如何创建几个0级变量

In [ ]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import division

import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # any {'0', '1', '2'} tensorflow suppress warnings

import numpy as np
import tensorflow as tf

In [ ]:
mammal = tf.Variable("Elephant", tf.string)
ignition = tf.Variable(451, tf.int16)
floating = tf.Variable(3.14159265359, tf.float64)
its_complicated = tf.Variable(12.3 - 4.85j, tf.complex64)

> 注：字符串在TensorFlow中被视为单个项目，而不是字符序列。可以有标量字符串，字符串向量等。

### Rank 1

要创建ranke为1的tf.Tensor对象，您可以传递一个list作为初始值。例如：

In [ ]:
mystr = tf.Variable(["Hello"], tf.string)
cool_numbers  = tf.Variable([3.14159, 2.71828], tf.float32)
first_primes = tf.Variable([2, 3, 5, 7, 11], tf.int32)
its_very_complicated = tf.Variable([12.3 - 4.85j, 7.5 - 6.23j], tf.complex64)

### 更高的Rank

秩为2的tf.Tensor对象由至少一行和至少一列组成：

In [ ]:
mymat = tf.Variable([[7],[11]], tf.int16)
myxor = tf.Variable([[False, True],[True, False]], tf.bool)
linear_squares = tf.Variable([[4], [9], [16], [25]], tf.int32)
squarish_squares = tf.Variable([ [4, 9], [16, 25] ], tf.int32)
rank_of_squares = tf.rank(squarish_squares)
mymatC = tf.Variable([[7],[11]], tf.int32)

更高级的张量同样由一个n维数组组成。例如，在图像处理过程中，会使用许多秩为4的张量，尺寸对应批量示例，图像宽度，图像高度和色彩通道。

In [ ]:
my_image = tf.zeros([10, 299, 299, 3])  # batch x height x width x color

### 获取tf.Tensor对象的秩

要确定tf.Tensor对象的秩，请调用tf.rank方法。例如，以下方法以编程方式确定上一节中定义的tf.Tensor的排名：

In [ ]:
r = tf.rank(my_image)
# After the graph runs, r will hold the value 4.

### 访问 tf.Tensor 切片(片段)

由于tf.Tensor是一个n维单元阵列，要访问tf.Tensor中的单个单元格，您需要指定索引n。

对于秩0张量（一个标量），索引是不必要的，因为它已经是一个单一的数字。 

对于秩1张量（矢量），传递单个索引允许您访问一个数字：

In [ ]:
my_scalar = first_primes[2]

请注意，在`[]`中传递的索引本身可以是一个标量tf.Tensor，如果您想从向量中动态选择一个元素。

对于秩2的tf.Tensor，传递两个数字会返回一个标量，如预期的那样：

In [ ]:
my_scalar = squarish_squares[1, 1]

但是，传递一个数字将返回一个矩阵的子向量，如下所示：

In [ ]:
my_row_vector = squarish_squares[1]
my_column_vector = squarish_squares[:, 1]

`：`符号是python切片语法，表示“不用管这个维度”。这在更高等级的张量中很有用，因为它允许你访问它的子向量，子矩阵，甚至其他的副本。

## Shape

张量的形状是每个维度中元素的数量。 TensorFlow在图形构建过程中自动推断形状。这些推断的形状可能具有已知或未知的秩。如果秩已知，则每个维度的大小可能是已知的或未知的。

TensorFlow文档使用三种符号约定来描述张量维度：rank, shape, dimension number。下表显示了它们如何相互关联：

|Rank|	Shape|	Dimension number|	Example|
|----|-------|------------------|----------|
|0 |  [] | 0-D | A 0-D tensor. A scalar. |
|1 |  [D0] |    1-D | A 1-D tensor with shape [5]. |
|2 |  [D0, D1] |    2-D | A 2-D tensor with shape [3, 4]. |
|3 |  [D0, D1, D2] |   3-D | A 3-D tensor with shape [1, 4, 3]. |
|n |  [D0, D1, ... Dn-1] | n-D | A tensor with shape [D0, D1, ... Dn-1]. |

### 获取tf.Tensor对象的形状

有两种方法可以访问tf.Tensor的形状。在构建图形时，询问已知的张量形状通常很有用。这可以通过读取tf.Tensor对象的shape属性来完成。此方法返回一个TensorShape对象，这是一种表示部分指定形状的便捷方式（因为在构建图形时并不会完全知道所有形状）。

也可以获得一个tf.Tensor，它将在运行时代表另一个tf.Tensor的完全定义时的形状。这是通过调用tf.shape操作完成的。这样，您可以构建一个图形，通过构建其他张量来控制张量的形状，这些张量取决于输入tf.Tensor的动态形状。

例如，下面是如何定义与给定矩阵中的列数相同大小的零向量：

In [ ]:
zeros = tf.zeros(squarish_squares.shape[1])

### 改变tf.Tensor的形状

张量元素的数量是其所有维度大小的乘积。标量的元素数量始终为1. 由于通常有许多不同的形状具有相同数量的元素，因此能够更改tf.Tensor的形状通常很方便，并保持元素的固定。这可以用tf.reshape完成。

以下示例演示如何reshape张量：

In [ ]:
rank_three_tensor = tf.ones([3, 4, 5])
matrix = tf.reshape(rank_three_tensor, [6, 10])  # Reshape existing content into
                                                 # a 6x10 matrix
matrixB = tf.reshape(matrix, [3, -1])  #  Reshape existing content into a 3x20
                                       # matrix. -1 tells reshape to calculate
                                       # the size of this dimension.
matrixAlt = tf.reshape(matrixB, [4, 3, -1])  # Reshape existing content into a
                                             #4x3x5 tensor

# Note that the number of elements of the reshaped Tensors has to match the
# original number of elements. Therefore, the following example generates an
# error because no possible value for the last dimension will match the number
# of elements.

# yet_another = tf.reshape(matrixAlt, [13, 2, -1])  # ERROR!

## Data types

除维度外，张量具有数据类型。

不可能拥有超过一种数据类型的tf.Tensor。但是，可以将任意数据结构作为字符串序列化并将其存储在tf.Tensors中。

使用tf.cast可以将tf.Tensors从一个数据类型转换为另一个数据类型：

In [ ]:
# Cast a constant integer tensor into floating point.
float_tensor = tf.cast(tf.constant([1, 2, 3]), dtype=tf.float32)

要检查tf.Tensor的数据类型，请使用Tensor.dtype属性。

从python对象创建tf.Tensor时，您可以选择指定数据类型。如果你不这样做，TensorFlow会选择一个可以表示数据的数据类型。 
**TensorFlow将Python整数转换为tf.int32并将python浮点数转换为tf.float32**。
否则，TensorFlow使用numpy在转换为数组时使用的相同规则。

### Evaluating Tensors

一旦构建了计算图，您可以运行生成特定tf.Tensor的计算并获取分配给它的值。这对于调试以及大部分TensorFlow工作都非常有用。

评估张量的最简单方法是使用Tensor.eval方法。例如：

In [ ]:
constant = tf.constant([1, 2, 3])
tensor = constant * constant
with tf.Session() as sess:
    print(tensor.eval())

Tensor.eval返回一个与张量的内容相同的numpy数组。

有时候没有上下文的时候不可能评估tf.Tensor，因为它的值可能取决于不可用的动态信息。例如，如果不为占位符提供值，则无法评估依赖于占位符的张量。

In [ ]:
p = tf.placeholder(tf.float32)
t = p + 1.0
t.eval()  # This will fail, since the placeholder did not get a value.
t.eval(feed_dict={p:2.0})  # This will succeed because we're feeding a value
                           # to the placeholder.

> 请注意，可以feed任何类型的tf.Tensor，而不仅仅是占位符。

其他模型构造可能会使评估tf.Tensor变得复杂。 TensorFlow无法直接评估函数内部或控制流结构内部定义的tf.Tensors。
如果一个tf.Tensor依赖于一个队列中的值，评估tf.Tensor只有在某个东西被排队后才会工作;
否则，评估它将挂起。在处理队列时，请记住在评估任何tf.Tensors之前调用tf.train.start_queue_runners。


### 打印Tensors

出于调试目的，您可能想要打印tf.Tensor的值。虽然tfdbg提供高级调试支持，但TensorFlow也可以直接打印tf.Tensor的值。

```python
t = <<some tensorflow operation>>
print t  # This will print the symbolic tensor when the graph is being built.
         # This tensor does not have a value in this context.
```

此代码打印tf.Tensor对象（代表延迟计算）而不是其值。相反，TensorFlow提供了tf.Print op，该操作将其第一个张量参数原封不动返回，同时会将第二个参数中传递的tf.Tensors集合打印出来。

要正确使用tf.Print，它的返回值必须被使用。看下面的例子

```python
t = <<some tensorflow operation>>
tf.Print(t, [t])  # This does nothing
t = tf.Print(t, [t])  # Here we are using the value returned by tf.Print
result = t + 1  # Now when result is evaluated the value of `t` will be printed.
```

当您评估`result`时，您将评估`result`依赖的所有内容。由于`result`取决于t，而评估t会产生打印其输入（旧的t值）的副作用，所以t会被打印。